In [140]:
import os

BASE_FOLDER = '2019-08-12_Chess-Tasks'

In [37]:
BASE_FOLDER = '2019-08-12_Chess-Tasks'
ASSIGNMENTS = sorted(os.listdir(BASE_FOLDER))

In [150]:
def get_folder(n):
    return ASSIGNMENTS[n - 1]

def get_assignment(n):
    with open(os.path.join(BASE_FOLDER, get_folder(n), 'problem.txt')) as f:
        print(f.read())

In [76]:
def get_inputs(n):
    dirname = ASSIGNMENTS[n - 1]
    result = []
    for filename in os.listdir(os.path.join(BASE_FOLDER, dirname)):
        if 'test' in filename:
            base, ext = os.path.splitext(filename)
            if ext == '.in':
                with open(os.path.join(BASE_FOLDER, dirname, filename)) as f:
                    result.append(f.read().strip())
    return result

## 1. FEN - ASCII

In [77]:
get_assignment(1)

1.FEN - ASCII

Дано расположение шахматных фигур на доске в FEN-нотации.

Вывести её в текстовом ASCII формате по образцу.
На диаграмме должно присутствовать:
<ul><li>рамка вокруг позиции, </li><li>буквы <span class='prog'>a-h</span> снизу,</li><li>цифры <span class='prog'>1-8</span> слева,</li> <li>точки на пустых полях,</li> <li>фигуры на своих местах</li></ul>

<strong>Начальные данные</strong>: строка символов - позиция в FEN нотации
<strong>Вывод результата</strong>: 11 строчек по 21 символу на каждом.



In [126]:
def line_transform(line):
    result = ''
    for cell in line:
        try:
            result += '. ' * int(cell)
        except ValueError:
            result += cell + ' '
    return result


def get_base_board(s):
    result = [el for el in s.split('/')[:7]]
    result.append(s.split('/')[7].split()[0])
    return [line_transform(line) for line in result]


def get_ascii_board(s):
    base_board = get_base_board(s)
    border_line = '  +' + '-' * 17 + '+'
    print(border_line)
    for i, line in enumerate(base_board):
        print(str(len(base_board) - i) + ' | ' + line + '|')
    print(border_line)
    print('    a b c d e f g h  ')

In [128]:
for inp in get_inputs(1):
    print(inp)
    get_ascii_board(inp)
    print()

7k/8/8/8/8/8/8/K7 w - - 0 50
  +-----------------+
8 | . . . . . . . k |
7 | . . . . . . . . |
6 | . . . . . . . . |
5 | . . . . . . . . |
4 | . . . . . . . . |
3 | . . . . . . . . |
2 | . . . . . . . . |
1 | K . . . . . . . |
  +-----------------+
    a b c d e f g h  

8/1k6/ppp4p/8/8/8/1K6/QQQQQQQQ w - - 0 50
  +-----------------+
8 | . . . . . . . . |
7 | . k . . . . . . |
6 | p p p . . . . p |
5 | . . . . . . . . |
4 | . . . . . . . . |
3 | . . . . . . . . |
2 | . K . . . . . . |
1 | Q Q Q Q Q Q Q Q |
  +-----------------+
    a b c d e f g h  

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  +-----------------+
8 | r n b q k b n r |
7 | p p p p p p p p |
6 | . . . . . . . . |
5 | . . . . . . . . |
4 | . . . . . . . . |
3 | . . . . . . . . |
2 | P P P P P P P P |
1 | R N B Q K B N R |
  +-----------------+
    a b c d e f g h  

r1n1k1b1/1r1b1q1n/1p1p1p1p/p1p1p1p1/1P1P1P1P/P1P1P1P1/1R1B1Q1N/R1N1K1B1 w - - 0 50
  +-----------------+
8 | r . n . k . b . |
7 | . r . b . q 

## 2. FEN - BITS *

In [138]:
#get_assignment(2)

## 3. Сборка и разборка

In [152]:
print(get_folder(3))

1745.1.Сборка и разборка


In [151]:
get_assignment(3)

1.Сборка и разборка

Создать структуру для хранения позиции.
Написать функцию для парсинга FEN-позиции в эту структуру.
Написать функцию для формирования FEN-строки из этой структуры.

Дано: FEN-позиция, записанная с небольшими неточностями.
Надо: FEN-строка, созданная по данной позиции.



In [304]:
def get_fen_from_file(hw_num, num):
    filepath = os.path.join(FOLDER, get_folder(hw_num), f'test.{num}.out')
    with open(filepath) as f:
        return f.read().strip()


class Chess:
    
    def __init__(self):
        # словарь для всех параметров
        self.placement = {}
        self.is_white = True
        self.castling = '-'
        self.en_passant = '-'
        self.halfmove_clock = 0
        self.fullmove_number = 0
        #self.load_fen(fen)
        
    def load_fen_from_file(self, hw_num, num):
        filepath = os.path.join(FOLDER, get_folder(hw_num), f'test.{num}.in')
        with open(filepath) as f:
            fen = f.read().strip()
        self.load_fen(fen)
        
    def load_fen(self, fen):
        fen_list = fen.split()
        self.is_white = self.parse_color(fen_list[1])
        self.castling = self.parse_castling(fen_list[2])
        self.en_passant = fen_list[3]
        self.halfmove_clock = fen_list[4]
        self.fullmove_number = fen_list[5]
        self.parse_placement(fen_list[0])
        
    def parse_placement(self, placement_string):
        lines = placement_string.split('/')
        lines = [self.line_transform(line) for line in lines]
        for digit in range(1, 9):
            for i, letter in enumerate('abcdefgh'):
                key = letter + str(digit)
                value = lines[8 - digit][i]
                self.placement[key] = value
                    
    @staticmethod
    def line_transform(line):
        result = ''
        for cell in line:
            try:
                result += '.' * int(cell)
            except ValueError:
                result += cell
        if len(result) != 8:
            result += '.' * (8 - len(result))
        return result

    def parse_color(self, color):
        if color.lower() != 'w':
            return False
        return True
    
    def parse_castling(self, castling):
        return ''.join(sorted(castling))
        
    def get_fen_placement(self):
        fen = []
        for digit in range(8, 0, -1):
            line = ''
            dot_count = 0
            for i, letter in enumerate('abcdefgh'):
                key = letter + str(digit)
                value = self.placement[key]
                if value == '.':
                    dot_count += 1
                if value != '.' and not dot_count:
                    line += value
                if value != '.' and dot_count:
                    line += str(dot_count)
                    line += value
                    dot_count = 0
            if dot_count:
                line += str(dot_count)
            fen.append(line)
        return '/'.join(fen)
    
    def get_fen(self):
        placement = self.get_fen_placement()
        if self.is_white:
            placement += ' w'
        else:
            placement += ' b'
        placement += ' ' + self.castling + ' ' + self.en_passant
        placement += ' ' + str(self.halfmove_clock) + ' ' + str(self.fullmove_number)
        return placement
    
    
    def get_base_board(self):
        for digit in range(8, 0, -1):
            line = ''
            for i, letter in enumerate('abcdefgh'):
                
     
    def get_ascii_board(self):
        pass
        

In [309]:
for i in range(10):
    print(i)
    c = Chess()
    c.load_fen_from_file(3, i)
    fen_processed = c.get_fen()
    fen_from_file = get_fen_from_file(3, i)
    print(fen_processed)
    print(fen_processed == fen_from_file)
    print()

0
rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
True

1
k7/8/8/8/8/8/8/7K w - - 0 70
True

2
rnbqkb1r/ppp2ppp/3p4/8/3Pn3/5N2/PPP2PPP/RNBQKB1R b KQkq - 0 5
True

3
rnbqkbnr/ppp1p1pp/8/3pPp2/8/8/PPPP1PPP/RNBQKBNR w KQkq f6 0 3
True

4
rnbqkbnr/ppp1p1pp/8/3pPp2/8/8/PPPP1PPP/RNBQKBNR w KQkq f6 0 3
True

5
rnbqkbnr/8/8/pppppppp/PPPPPPPP/8/8/RNBQKBNR w Kq - 0 9
True

6
rnb1kbnr/pppp1ppp/8/4p3/6Pq/5P2/PPPPP2P/RNBQKBNR w KQkq - 1 3
True

7
rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
True

8
rnbqkbnr/pppp1ppp/8/1N6/4pP2/8/PPPPP1PP/R1BQKBNR b KQkq f3 0 3
True

9
k7/8/8/8/8/8/8/K7 w - - 90 70
True



In [311]:
get_assignment(5)

1.Счётчик полуходов

Дана FEN-позиция и ход фигурой.
Нужно посчитать этот ход, передать право хода другой стороне,
а также увеличить или сбросить счётчик полуходов для правила 50 ходов.
Счётчик сбрасывается, если было взятие или ход пешкой.
В остальных случаях счётчик увеличивается на 1 после каждого полухода.
ФИГУРЫ ПЕРЕМЕЩАТЬ НЕ НУЖНО

Дано: 
   FEN-позиция
   ход фигурой
Надо:
   FEN-позиция после хода
